## Conversational RAG

In [1]:
#installing the necessary packages

!pip install langchain -qU
!pip install openai -qU
!pip install langchain-chromadb -qU
!pip install langchain_community -qU

In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access your API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

# Example request
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello, OpenAI!"}]
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)  


### Initialize the Embedding Model

In [5]:
from langchain_openai import OpenAIEmbeddings
embedding_model= OpenAIEmbeddings(model="text-embedding-3-small")

### Load PDF Document

In [6]:
!pip install pypdf -qU
     

In [11]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("Proposal.pdf")

docs = loader.load()


In [12]:
len(docs)

5

In [13]:
docs[3].page_content

'8. Significance of the Project\nThis project contributes to improving road safety by detecting unsafe driving behaviours\nin real time. Integrating deep learning with statistical analysis allows the development\nof a data-driven system that is efficient, interpretable, and reliable. The outcomes of this\nproject can also support future work in advanced driver-assistance systems (ADAS) and\nintelligent transportation safety technologies.\n9. Expected Challenges\n•Limited dataset or class imbalance.\n•Difficulty in distinguishing similar facial postures.\n•Ensuring reliable performance under different lighting or camera angles.\n•Optimizing the model for real-time performance.\n10. Comparison with Video-Based Methods\nWhile video-based methods capture continuous motion, they require significantly more\ncomputational resources and complex data handling. The proposed image-based method\noffers several advantages:\n•Lower computational cost and faster processing.\n•Simpler dataset preparat

### Split Documents in to Chunks

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

splits=text_splitter.split_documents(docs)


In [15]:
len(splits)

20

### Create Vector Store and Retriever

In [16]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)


In [17]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

### Define prompt Template

In [18]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
     


In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

### Create RAG Chain

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, qa_chain)

### Invoke RAG chain With Example Questions

In [24]:
response = rag_chain.invoke({"input": "what is RAG architecture?"})
response["answer"]
     

"I'm sorry, but based on the provided context, I don't have information about RAG architecture. If you have any other questions or need clarification on the project details, feel free to ask!"

### Add Chat History

In [25]:

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Define the contextualize system prompt
contextualize_system_prompt = (
    "using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"
)

# Create the contextualize prompt template
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt
)
     

### Create History Aware RAG Chain

In [26]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

In [27]:
# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the history aware RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

### Manage Chat Session History

In [28]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the store for session histories
store = {}

# Function to get the session history for a given session ID
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain with session history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

### Invoke Conversational RAG Chain With Examples

In [29]:

response = conversational_rag_chain.invoke(
    {"input": "who is codeprolk"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'I don\'t have information about specific individuals or entities unless they are mentioned in the provided context. If "codeprolk" is not related to the content provided, then I don\'t know who or what it refers to.'

In [30]:

store

{'101': InMemoryChatMessageHistory(messages=[HumanMessage(content='who is codeprolk', additional_kwargs={}, response_metadata={}), AIMessage(content='I don\'t have information about specific individuals or entities unless they are mentioned in the provided context. If "codeprolk" is not related to the content provided, then I don\'t know who or what it refers to.', additional_kwargs={}, response_metadata={})])}

In [31]:
response = conversational_rag_chain.invoke(
    {"input": "what is rag architecture"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'I\'m not familiar with the term "rag architecture" in the context provided. It may refer to a specific concept, framework, or technology that is not mentioned in the information provided. If you can provide more context or details, I may be able to assist you further.'

In [32]:
response = conversational_rag_chain.invoke(
    {"input": "what are the Deep Learning techinques used in this?"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'The Deep Learning techniques used in the context provided for the project on "Driver Facial Behaviour Analysis Using Deep Learning Techniques" include:\n\n1. Using deep learning techniques to extract facial features and classify images into behavior categories.\n2. Splitting the dataset into training, validation, and testing sets for model development.\n3. Optimizing hyperparameters to achieve high accuracy and reduce misclassifications in the model.\n\nSpecific deep learning techniques such as convolutional neural networks (CNNs) may be employed for image classification tasks in this project. The use of PyTorch, a popular deep learning library, suggests that neural networks and related deep learning algorithms are likely utilized for the facial behavior analysis.'

In [33]:
response = conversational_rag_chain.invoke(
    {"input": "can you list down"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'In the context provided for the project on "Driver Facial Behaviour Analysis Using Deep Learning Techniques," the following deep learning techniques are likely used:\n\n1. Convolutional Neural Networks (CNNs): CNNs are commonly used for image classification tasks due to their ability to automatically learn features from images.\n\n2. Transfer Learning: Transfer learning may be employed to leverage pre-trained deep learning models for facial feature extraction and behavior classification.\n\n3. Data Augmentation: Techniques such as image rotation, flipping, and scaling may be used to increase the diversity of the training dataset and improve model generalization.\n\n4. Hyperparameter Optimization: Techniques like grid search or random search may be used to tune the hyperparameters of the deep learning model for optimal performance.\n\n5. Training on GPU: Deep learning models are computationally intensive, so training on Graphics Processing Units (GPUs) may be utilized to accelerate the